In [1]:
import sqlite3

connection=sqlite3.connect("vodacom_demo.db")

print("Connected to Database")

Connected to Database


In [5]:
cursor=connection.cursor()

# students 

query='''
CREATE TABLE IF NOT EXISTS students(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    age INTEGER,
    grade TEXT
)
'''

cursor.execute(query)
print("Table Created Successfully")

Table Created Successfully


#### Insert into the Table

In [9]:
query="INSERT INTO students (name,age,grade) values ('Tom',25,'A')"
cursor.execute(query)
connection.commit()

In [11]:
query="INSERT INTO students (name,age,grade) values ('Elizabeth',24,'B')"
cursor.execute(query)
connection.commit()

In [13]:
query="INSERT INTO students (name,age,grade) values ('Mike',26,'C')"
cursor.execute(query)
connection.commit()

#### Query the data from database

In [68]:
query="Select * from students"
cursor.execute(query)
rows=cursor.fetchall()

print(type(rows))

for row in rows:
    print(type(row))
    print(row)

<class 'list'>
<class 'tuple'>
(1, 'Tom', 25, 'A')
<class 'tuple'>
(3, 'Elizabeth', 24, 'A+')
<class 'tuple'>
(4, 'Mike', 26, 'C')


#### Update and Delete the records from Database

#### Update

In [27]:
query="Update students set grade='A+' WHERE name='Elizabeth'"
cursor.execute(query)
connection.commit()

#### Delete

In [32]:
query ="Delete from students where id=2"
cursor.execute(query)
connection.commit()

## INSTALLING SQLALCHEMY

In [39]:
!pip install sqlalchemy

In [43]:
from sqlalchemy import create_engine

database_url="sqlite:///vodacom_test.db"
engine=create_engine(database_url,echo=True)

print("Engine generated successfully")

Engine generated successfully


### Define the Table using ORM[Object Relational Mapping]

In [46]:
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column,Integer,String

Base=declarative_base()

class Customer(Base):
    __tablename__="customers"

    id=Column(Integer,primary_key=True,autoincrement=True)
    name=Column(String(100),nullable=False)
    email=Column(String(100),nullable=False,unique=True)
    age=Column(Integer,nullable=True)

    def __repr__(self):
        return f"Customer [Id: {self.id} | Name:{self.name} | Email : {self.email} | Age: {self.age}]"

#### CREATE THE DATABASE TABLE 


In [49]:
Base.metadata.create_all(engine)
print("Tables created")

2024-10-17 19:01:47,742 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-17 19:01:47,743 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("customers")
2024-10-17 19:01:47,743 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-17 19:01:47,744 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("customers")
2024-10-17 19:01:47,745 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-17 19:01:47,746 INFO sqlalchemy.engine.Engine 
CREATE TABLE customers (
	id INTEGER NOT NULL, 
	name VARCHAR(100) NOT NULL, 
	email VARCHAR(100) NOT NULL, 
	age INTEGER, 
	PRIMARY KEY (id), 
	UNIQUE (email)
)


2024-10-17 19:01:47,747 INFO sqlalchemy.engine.Engine [no key 0.00075s] ()
2024-10-17 19:01:47,769 INFO sqlalchemy.engine.Engine COMMIT
Tables created


#### Insert record into table

In [52]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session=Session()

customer1=Customer(name="Tom Smith",age=40,email="tom.smith@gmail.com")
customer2=Customer(name="Penny Colin",age=34,email="penny.colin@gmail.com")

session.add(customer1)
session.add(customer2)

session.commit()

2024-10-17 19:12:01,498 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-17 19:12:01,500 INFO sqlalchemy.engine.Engine INSERT INTO customers (name, email, age) VALUES (?, ?, ?) RETURNING id
2024-10-17 19:12:01,501 INFO sqlalchemy.engine.Engine [generated in 0.00015s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('TOm Smith', 'tom.smith@gmail.com', 40)
2024-10-17 19:12:01,503 INFO sqlalchemy.engine.Engine INSERT INTO customers (name, email, age) VALUES (?, ?, ?) RETURNING id
2024-10-17 19:12:01,503 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('Penny Colin', 'penny.colin@gmail.com', 34)
2024-10-17 19:12:01,505 INFO sqlalchemy.engine.Engine COMMIT


#### Query the data in db

In [66]:
customers=session.query(Customer).all()
print(type(customers))
for customer in customers:
    print(type(customer))
    print(customer)

2024-10-17 19:26:34,540 INFO sqlalchemy.engine.Engine SELECT customers.id AS customers_id, customers.name AS customers_name, customers.email AS customers_email, customers.age AS customers_age 
FROM customers
2024-10-17 19:26:34,540 INFO sqlalchemy.engine.Engine [cached since 518.9s ago] ()
<class 'list'>
<class '__main__.Customer'>
Customer [Id: 1 | Name:Thomas Smith | Email : tom.smith@gmail.com | Age: 39]
<class '__main__.Customer'>
Customer [Id: 2 | Name:Penny Colin | Email : penny.colin@gmail.com | Age: 34]


In [59]:
tom=session.query(Customer).filter_by(id=1).first()
print(tom)

2024-10-17 19:23:07,148 INFO sqlalchemy.engine.Engine SELECT customers.id AS customers_id, customers.name AS customers_name, customers.email AS customers_email, customers.age AS customers_age 
FROM customers 
WHERE customers.id = ?
 LIMIT ? OFFSET ?
2024-10-17 19:23:07,149 INFO sqlalchemy.engine.Engine [generated in 0.00084s] (1, 1, 0)
Customer [Id: 1 | Name:TOm Smith | Email : tom.smith@gmail.com | Age: 40]


#### Update record in Alchemy

In [70]:
tom=session.query(Customer).filter_by(id=1).first()
print(tom)

tom.name="Thomas Smith"
tom.age=39

session.commit()

2024-10-17 19:27:39,186 INFO sqlalchemy.engine.Engine SELECT customers.id AS customers_id, customers.name AS customers_name, customers.email AS customers_email, customers.age AS customers_age 
FROM customers 
WHERE customers.id = ?
 LIMIT ? OFFSET ?
2024-10-17 19:27:39,187 INFO sqlalchemy.engine.Engine [cached since 272s ago] (1, 1, 0)
Customer [Id: 1 | Name:Thomas Smith | Email : tom.smith@gmail.com | Age: 39]
2024-10-17 19:27:39,188 INFO sqlalchemy.engine.Engine COMMIT


#### Delete the record

In [73]:


penny=session.query(Customer).filter_by(id=2).first()
print(penny)

session.delete(penny)
session.commit()

print("Record Deleted")

2024-10-17 19:31:19,700 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-17 19:31:19,701 INFO sqlalchemy.engine.Engine SELECT customers.id AS customers_id, customers.name AS customers_name, customers.email AS customers_email, customers.age AS customers_age 
FROM customers 
WHERE customers.id = ?
 LIMIT ? OFFSET ?
2024-10-17 19:31:19,701 INFO sqlalchemy.engine.Engine [cached since 492.6s ago] (2, 1, 0)
Customer [Id: 2 | Name:Penny Colin | Email : penny.colin@gmail.com | Age: 34]
2024-10-17 19:31:19,703 INFO sqlalchemy.engine.Engine DELETE FROM customers WHERE customers.id = ?
2024-10-17 19:31:19,704 INFO sqlalchemy.engine.Engine [generated in 0.00050s] (2,)
2024-10-17 19:31:19,705 INFO sqlalchemy.engine.Engine COMMIT
Record Deleted
